In [ ]:
"""features: 
- average sentence length (in words)
- average review length (in words)
- average review length (in sentences)
- paragraph rate
- bulleted or numbered list rate
- all caps, bad punctuation, run on sentences?
- bag of words: common words in elite vs. not elite; fp, fn, etc. 
"""

In [296]:
import string, re
import math
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from spacy.en import English, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, tqdm_pandas

In [2]:
nlp = English()

In [767]:
d = nltk.Text(b)

In [467]:
reviews = pd.read_csv('data/yelp_academic_dataset_review.csv')

In [87]:
stop = STOPWORDS
punct = {p for p in string.punctuation}

### get descriptive features of review text

In [258]:
def get_num_words(text):
    """Get number of words per review."""
    return float(len(text.split()))

In [257]:
# no spacy
def get_num_sents(text):
    """Get number of sentences per review."""
    # add 1 at the end for last punctuation 
    return text.count('. ') + text.count('! ') + text.count('? ') + text.count(') ') + \
            text.count('.\n') + text.count('!\n') + text.count('?\n') + text.count(')\n') + 1.0

In [259]:
def get_num_para(text):
    """Get number of paragraphs per review."""
    return text.count('\n\n') + 1.0

In [929]:
def mentions_price(text):
    """Check if review mentions price ($). Return 1 if yes, 0 if no."""
    return 1 if '$' in text else 0

In [977]:
def get_allcaps(text):
    """Get number of all uppercase words in review."""
    text = re.sub("[^a-zA-Z]", " ", text)
    return len([word for word in text.split() if word.isupper() and len(word) > 2])

In [941]:
def get_exclamations(text):
    """Get number of exclamation marks in review."""
    return text.count('!')

In [153]:
def get_num_chars(text):
    return float(len([char for char in text if char != ' ' and char not in punct]))

In [81]:
reviews.columns

Index([         u'user_id',        u'review_id',             u'text',
             u'votes.cool',      u'business_id',      u'votes.funny',
                  u'stars',             u'date',             u'type',
           u'votes.useful',  u'review_len_wrds',  u'review_len_sent',
       u'avg_wrds_in_sent',         u'num_para',   u'mentions_price',
            u'num_allcaps', u'num_exclamations',           u'tokens',
               u'is_elite'],
      dtype='object')

In [79]:
t = reviews.loc[0, 'text']

In [80]:
t

u'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.'

In [ ]:
# bulleted or numbered list 

In [742]:
def decode(text):
    try:
        return text.decode('utf8')
    except:
        return text

In [866]:
# with spacy
def get_num_sents_spacy(text):
    try:
        return len([sent for sent in nlp(text).sents])
    except:
        return

### get tokens -- *not currently implemented*

In [476]:
def get_clean_tokens(text):
    #letters_only = re.sub("[^a-zA-Z]", " ", text) 
    union = punct.union(stop)
    #spacing = {'', ' ', '\n', '\n\n'}
    tokens = [token.lemma_ for token in nlp(text.decode('utf8'))]
    filtered = [token for token in tokens if token not in union]
    while "" in filtered:
        filtered.remove("")
    while " " in filtered:
        filtered.remove(" ")
    while "\n" in filtered:
        filtered.remove("\n")
    while "\n\n" in filtered:
        filtered.remove("\n\n")
    return ' '.join(filtered)

In [613]:
# version without utf decoding
# def get_clean_tokens2(text):  
#     letters_only = re.sub("[^a-zA-Z]", " ", text) 
#     words = ' '.join(letters_only.lower().split())
#     tokens = [token.lemma_ for token in nlp(words)]
#     filtered = [t for t in tokens if t not in stop and t != '' and t != ' ' and t != '\n' and t != '\n\n']
#     return ' '.join(filtered)

In [1014]:
# uses utf decoding
def get_clean_tokens2(text):  
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
    words = ' '.join(letters_only.lower().split())
    tokens = [token.lemma_ for token in nlp(words)]
    filtered = [t for t in tokens if t not in stop and t != '' and t != ' ' and t != '\n' and t != '\n\n']
    return ' '.join(filtered)

In [1032]:
def tokenize2(df):
    return map(get_clean_tokens2, df.text.values)  

In [1067]:
def tokenize4(df):
    tokens = []
    for i in tqdm(range(len(df.text.values))):
        tokens.append(get_clean_tokens2(df.text.values[i]))
    return tokens

In [1071]:
clntkns = tokenize4(reviews)

In [1075]:
with open('pickled/tokens.pkl', 'w') as picklefile:
    pickle.dump(clntkns, picklefile)

### parse reviews df

In [279]:
### OLD
def get_features_old(df):
    # get number of words in single review
    df.loc[:,'review_len_wrds'] = df.loc[:,'text'].apply(lambda x: len(cleantext(x)))
    
    # get number of sentences in single review
    df.loc[:,'review_len_sent'] = df.loc[:,'text'].apply(
        lambda x: len([sent for sent in nlp(x.decode('utf8')).sents])) # better way?
    
    # get average number of words per sentence 
    df.loc[:,'avg_wrds_in_sent'] = df.loc[:,'review_len_wrds'] / df.loc[:,'review_len_sent']
    
    # get cleaned tokens for bag of words
    df.loc[:,'clean_tkns'] = df.loc[:, 'text'].apply(lambda x: get_clean_tokens(x))

In [969]:
def get_features(df):
#     decode
    df.loc[:, 'text'] = df.loc[:, 'text'].apply(decode)

    # get number of words in single review
    df.loc[:,'review_len_wrds'] = df.loc[:,'text'].apply(get_num_words)

    #get number of sentences in single review
    df.loc[:,'review_len_sent'] = df.loc[:,'text'].apply(get_num_sents)

    # get average number of words per sentence 
    df.loc[:,'avg_wrds_in_sent'] = df.loc[:,'review_len_wrds'] / df.loc[:,'review_len_sent']
    
    # get number of paragraphs
    df.loc[:, 'num_para'] = df.loc[:, 'text'].apply(get_num_para)
    
    # check if price is mentioned
    df.loc[:, 'mentions_price'] = df.loc[:, 'text'].apply(mentions_price)
    
    # get number of all caps words
    df.loc[:, 'num_allcaps'] = df.loc[:, 'text'].apply(get_allcaps)
    
    # get number of exclamation marks
    df.loc[:, 'num_exclamations'] = df.loc[:, 'text'].apply(get_exclamations)

#     get cleaned tokens for bag of words
#     %time df.loc[:,'clean_tkns'] = df.loc[:, 'text'].apply(lambda x: get_clean_tokens2(x))

In [190]:
# additional stuff - add to previous cell when cleaning
def get_more_features(df):
    df.loc[:,'num_chars'] = df.loc[:,'text'].apply(get_num_chars)
    df.loc[:,'ari_score'] = df.apply(
        lambda row: 4.71 * (row.num_chars/float(row.review_len_wrds)) \
        + 0.5 * (row.review_len_wrds/float(row.review_len_sent)) - 21.43, 
        axis = 1)
    
"""
ARI (automatic readability index) score:
where characters is the number of letters and numbers, words is the number of spaces, 
and sentences is the number of sentences. 
"""

'\nARI (automatic readability index) score:\nwhere characters is the number of letters and numbers, words is the number of spaces, \nand sentences is the number of sentences. \n'

In [282]:
def get_chars_per_word(df):
    df['avg_chars_per_word'] = df.loc[:,'num_chars'] / df.loc[:,'review_len_wrds']

In [378]:
# add elite/nonelite words

def get_elite_words(tokens):
    try:
        tokens = tokens.split()
        return len(elite_words.intersection(tokens))
    except:
        return 0

def get_nonelite_words(tokens):
    try:
        tokens = tokens.split()
        return len(ne_words.intersection(tokens))
    except:
        return 0

In [382]:
def add_content_counts(df):
    df['num_elite_words'] = df.text.apply(get_elite_words)
    df['num_ne_words'] = df.text.apply(get_nonelite_words)

In [263]:
get_chars_per_word(reviews)
get_features(reviews)
get_more_features(reviews)

In [383]:
add_content_counts(reviews)

In [388]:
byuser = reviews.groupby('user_id')

In [390]:
user_content = byuser.mean().loc[:, 'num_elite_words':'num_ne_words']

In [210]:
user_ari = byuser.mean().loc[:,'ari_score']

In [289]:
user_wrd_length = byuser.mean().loc[:,'avg_chars_per_word']

In [1003]:
user_avgs = byuser.mean().loc[:, 'review_len_wrds':]

In [392]:
# pickle user words/content data
with open('pickled/user_content.pkl', 'w') as picklefile:
    pickle.dump(user_content, picklefile)

In [1009]:
# pickle user avgs data
with open('pickled/user_avgs.pkl', 'w') as picklefile:
    pickle.dump(user_avgs, picklefile)

In [214]:
# pickle user avg ari data
with open('pickled/user_ari.pkl', 'w') as picklefile:
    pickle.dump(user_ari, picklefile)

In [292]:
# pickle user avg word length data
with open('pickled/user_wrdlength.pkl', 'w') as picklefile:
    pickle.dump(user_wrd_length, picklefile)

In [1077]:
reviews['tokens'] = clntkns

In [293]:
with open('pickled/reviewsdf2.pkl', 'w') as picklefile:
    pickle.dump(reviews, picklefile)

In [3]:
with open('pickled/reviewsdf2.pkl', 'r') as picklefile:
    reviews = pickle.load(picklefile)

### bag of words, etc. workspace 

In [21]:
with open('pickled/users_elite.pkl', 'r') as picklefile:
    userids = pickle.load(picklefile)
    
userids.set_index('user_id', inplace = True)
reviews['is_elite'] = reviews.user_id.apply(lambda x: userids.loc[x, 'is_elite'])

split into train and test

In [308]:
train, test = train_test_split(reviews, test_size = .25)

In [66]:
elite_reviews = train[train.is_elite == 1]
nonelite_reviews = train[train.is_elite == 0]

define word vector parameters

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None, \
                             max_features = 500) 

get most popular tokens for elite users

In [67]:
elite_features = vectorizer.fit_transform(elite_reviews.tokens)
elite_words = vectorizer.get_feature_names()
elite_features = elite_features.toarray()
elite_dist = np.sum(elite_features, axis = 0)

In [68]:
elite_sorted = sorted(zip(elite_words, elite_dist), key = lambda x: x[1], reverse = True)
elite_wrds_dict = dict(elite_sorted)

In [305]:
len(elite_wrds_dict)

500

In [ ]:
onlyelite_words = {
    word : elite_wrds_dict[word] 
    for word in elite_wrds_dict 
    if word not in nonelite_wrds_dict}

In [341]:
# top 20 words used by elites but not non-elites
elite_top50 = sorted(onlyelite_words.items(), key = lambda x: x[1], reverse = True)[:50]

In [344]:
elite_words = {item[0] for item in elite_top50}

get popular tokens for non-elite users

In [70]:
nonelite_features = vectorizer.fit_transform(nonelite_reviews.tokens)
nonelite_words = vectorizer.get_feature_names()
nonelite_features = nonelite_features.toarray()
nonelite_dist = np.sum(nonelite_features, axis = 0)
nonelite_sorted = sorted(zip(nonelite_words, nonelite_dist), key = lambda x: x[1], reverse = True)

In [71]:
nonelite_wrds_dict = dict(nonelite_sorted)

In [ ]:
onlynonelite_words = {
    word : nonelite_wrds_dict[word] 
    for word in nonelite_wrds_dict 
    if word not in elite_wrds_dict}

In [337]:
# top 50 words used by elites but not non-elites
ne_top50 = sorted(onlynonelite_words.items(), key = lambda x: x[1], reverse = True)[:50]

In [347]:
ne_words = {item[0] for item in ne_top50}

#### some other stuff

In [329]:
train2 = train.dropna(subset=['tokens'])

In [331]:
# test model?
features = vectorizer.fit_transform(train2.tokens)
words = vectorizer.get_feature_names()
features = features.toarray()

In [7]:
# # get counts for each word in corpus 
# dist = np.sum(features, axis = 0)
# sorted(zip(words, dist), key = lambda x: x[1], reverse = True)[:100]

[(u'place', 1347563),
 (u'good', 1237040),
 (u'food', 1206517),
 (u'great', 1039474),
 (u'time', 987742),
 (u'like', 977408),
 (u'come', 834873),
 (u'service', 815935),
 (u'order', 767823),
 (u'try', 597247),
 (u'love', 531152),
 (u've', 472226),
 (u'nice', 469437),
 (u'look', 456345),
 (u'want', 445065),
 (u'restaurant', 430650),
 (u'price', 426258),
 (u'eat', 421034),
 (u'best', 420215),
 (u'know', 397865),
 (u'think', 397360),
 (u'wait', 392371),
 (u'little', 368174),
 (u'drink', 354747),
 (u'staff', 342886),
 (u'people', 341297),
 (u'day', 340781),
 (u'ask', 331337),
 (u'thing', 330179),
 (u'friendly', 329725),
 (u'pretty', 315407),
 (u'room', 313836),
 (u'menu', 311748),
 (u'experience', 311092),
 (u'need', 308685),
 (u'tell', 308509),
 (u'chicken', 308354),
 (u'work', 308325),
 (u'night', 298637),
 (u'bar', 297164),
 (u'table', 296750),
 (u'definitely', 296024),
 (u'way', 289305),
 (u'better', 285811),
 (u'feel', 285268),
 (u'bad', 284377),
 (u'delicious', 273730),
 (u'star', 269

In [332]:
model = GaussianNB().fit(features, train2.is_elite)

In [333]:
test_features = vectorizer.fit_transform(test.tokens)

In [334]:
test_features = test_features.toarray()
test_pred = model.predict(test_features)

In [335]:
print accuracy_score(test.is_elite, test_pred)
print precision_score(test.is_elite, test_pred)
print recall_score(test.is_elite, test_pred)

0.718772469729
0.43614221322
0.512898933237


In [336]:
test.is_elite.value_counts()[0] / float(test.is_elite.shape[0])

0.75549519687077571

In [298]:
pred = model.predict(test_features)

In [300]:
print accuracy_score(train.is_elite, pred)
print precision_score(train.is_elite, pred)
print recall_score(train.is_elite, pred)

0.727847081121
0.448259404194
0.485176924527


In [252]:
pred = model.predict(features)
print 'acc:', accuracy_score(test.is_elite, pred)
print 'acc:', precision_score(test.is_elite, pred, average = None)
print 'acc:', recall_score(test.is_elite, pred, average = None)
print 'acc:', f1_score(test.is_elite, pred, average = None)

acc: 1.0
acc: [ 1.  1.]
acc: [ 1.  1.]
acc: [ 1.  1.]
